In [1]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import os
import math
global year
year=[]

def get_corp_code():
    url = "http://comp.fnguide.com/XML/Market/CompanyList.txt"
    
    '''requests.get(url) is a Python function call using 
    the requestslibrary to perform an HTTP GET request 
    to the specified URL. This function is commonly used 
    in web scraping and API interactions to retrieve data 
    from a web server.'''    
    resp = requests.get(url)
    
    #encoding of the HTTP response content. 
    resp.encoding = "utf-8-sig"
    '''Parse the JSON (JavaScript Object Notation) data from the response
    JSON data is structured, easy to read, and commonly used for exchanging 
    information between different applications or systems, especially 
    in web development and APIs.'''
    data = resp.json()
    comp = data['Co']
    df = pd.DataFrame(data=comp)
    '''gb==701(stock) / 752(ritz)/ 770(etf)/ 780(future)'''
    cond = df['gb'] == '701'
    df2 = df[cond].copy()
    return df2

In [3]:
def get_closing_accounts_day(code):
    url = f"https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{code}"
    selector = "#compBody > div.section.ul_corpinfo > div.corp_group1 > h2:nth-child(4)"
    resp = requests.get(url)
    html = resp.text 
#to create a BeautifulSoup object to parse an HTML document 
# using the lxml parser
    soup = BeautifulSoup(html, "lxml")
    tags = soup.select(selector)
    return tags[0].text
get_closing_accounts_day('005930')

'12월 결산'

In [32]:
def get_roic(code):
    url= f"http://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701"
    roic=[]
    try:
        dfs=pd.read_html(url)
        df=dfs[0]
        df.set_index(df.columns[0],inplace=True)
        tem2=df.filter(regex=".*12$").columns.tolist()
        
        for i in range(0,len(tem2)):
            year.append(tem2[i][2:4])

        tem=df.filter(regex=".*12$").filter(like='ROIC',axis=0)
        for i in range(0,tem.shape[1]):
            if tem.iloc[0,i]=='':
                roic.append(np.nan)
            else:
                roic.append(float(tem.iloc[0,i]))
    except:
        roic=np.nan
    return roic
get_roic('005930')

[13.2, 17.2, 23.9, 25.9]

In [33]:
def get_ev_ebitda(code):
    ev_ebitda=[]
    url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701"
    try:
        dfs = pd.read_html(url)
        df = dfs[3]
        df.set_index(df.columns[0], inplace=True)
        tem = df.filter(regex=".*12$").filter(like='EV/EBITDA', axis=0)
        for i in range(0,tem.shape[1]):
            if tem.iloc[0,i]=='':
                ev_ebitda.append(np.nan)
            else:
                ev_ebitda.append(float(tem.iloc[0,i]))
    except:
        ev_ebitda = np.nan
    return ev_ebitda
get_ev_ebitda('005930')

[6.6, 8.33, 6.23, 4.62]

In [66]:
if not os.path.exists(f"./data/scraping data"):
    os.makedirs(f"./data/scraping data")

if __name__ == "__main__":
    df = get_corp_code()
    columns = ['code', 'name', 'day']
    data=[]
    c=0
    for i in range(len(df)):
#ex > repeatation of this {"cd":"A157500", "nm":"TIGER 증권", "gb":"770"}
        data_tem = []
        s = df.iloc[i]
        acode = s['cd']
        code = acode[1:]
        name = s['nm']
        
        day = get_closing_accounts_day(code)
        roic = get_roic(code)
        ev_ebitda = get_ev_ebitda(code)

        
        data_tem.append(code)
        data_tem.append(name)
        data_tem.append(day)
        
        try:
            for j in range(0,len(roic)):
                data_tem.append(roic[j])
                if c==0:
                    columns.append('roic'+year[j])
            for j in range(0,len(ev_ebitda)):
                data_tem.append(ev_ebitda[j])
                if c==0:
                    columns.append('ev/ebitda'+year[j])
                    year_backup=year
            if len(year)!=0:
                c=1
        except:
            ()    
        data.append((data_tem))
        print(data)
        year=[]
        if (i+1) % 100 == 0:
            df1 = pd.DataFrame(data=data, columns=columns)
            df1.to_excel(f"./data/scraping data/data_{i+1}.xlsx")
            data = []
        time.sleep(0.000000001)
    # last companies            
    df1 = pd.DataFrame(data=data, columns=columns)
    df1.to_excel(f"./data/scraping data/data_last.xlsx")

[['060310', '3S', '03월 결산']]
[['060310', '3S', '03월 결산'], ['095570', 'AJ네트웍스', '12월 결산', 1.2, 1.5, 2.5, 6.1, 6.56, 5.73, 4.78, 4.53]]
[['060310', '3S', '03월 결산'], ['095570', 'AJ네트웍스', '12월 결산', 1.2, 1.5, 2.5, 6.1, 6.56, 5.73, 4.78, 4.53], ['006840', 'AK홀딩스', '12월 결산', 5.2, -7.2, -5.2, -0.8, 5.48, 39.71, 34.56, 11.39]]
[['060310', '3S', '03월 결산'], ['095570', 'AJ네트웍스', '12월 결산', 1.2, 1.5, 2.5, 6.1, 6.56, 5.73, 4.78, 4.53], ['006840', 'AK홀딩스', '12월 결산', 5.2, -7.2, -5.2, -0.8, 5.48, 39.71, 34.56, 11.39], ['054620', 'APS', '12월 결산', -378.8, -50.2, -8.1, -39.6, nan, nan, 274.52, nan]]


KeyboardInterrupt: 

In [42]:
!pip install finance-datareader

In [67]:
year=year_backup
# read excels and concat dataframe
xls=os.listdir(f"data/scraping data")
dfs=[]
for i in xls:
# dtype={"code":str} : sets the "code" column to be of type string (str)
    df=pd.read_excel(f"data/scraping data/{i}",index_col=0, dtype={"code":str})
    dfs.append(df)

In [68]:
import FinanceDataReader as fdr
cumulative_return=1
for i in range(0,len(year)-1):
    df=pd.concat(dfs)
    cond=df['day']=="12월 결산"
    df=df[cond]
    cond=(df['roic'+year[i]]>0)&(df['ev/ebitda'+year[i]]>0)
    df2=df[cond].copy()
    df2['rank1']=df2['roic'+year[i]].rank(ascending=False)
    df2['rank2']=df2['ev/ebitda'+year[i]].rank()
    df2['rank']=df2['rank1']+df2['rank2']
    df3=df2.sort_values(by='rank').head(n=30)
    Return_list=[]
    for j in range(0,df3.shape[0]):
        try:
            df=fdr.DataReader(symbol=df3.iloc[j,0])
            df1=df.loc[df.index.year == int('20'+year[i+1])]
            Return=(df1.iloc[-1]['Close']-df1.iloc[0]['Close'])/df1.iloc[0]['Close']
            Return_list.append(Return)
        except:
            ()
    print('return(',int(year[i+1]),') :',sum(Return_list)/len(Return_list))
    cumulative_return*=1+sum(Return_list)/len(Return_list)
print('magic formula_cumulative_return :',cumulative_return-1)
print('magic formula_return(CAGR) :',cumulative_return**(1/3)-1)

"003340" invalid symbol or has no data
return( 20 ) : 0.4709952933874966
"236010" invalid symbol or has no data
"166220" invalid symbol or has no data
"140320" invalid symbol or has no data
return( 21 ) : 0.8154104413900081
"146930" invalid symbol or has no data
"101910" invalid symbol or has no data
return( 22 ) : -0.26760051209227337
magic formula_cumulative_return : 0.9558436938349912
magic formula_return(CAGR) : 0.25057971911716415


In [69]:
df=fdr.DataReader(symbol='229200')
cumulative_kosdaq_return=1
for i in range(0,len(year)-1):
    df1=df.loc[df.index.year == int('20'+year[i+1])]
    kosdaq_return=(df1.iloc[-1]['Close']-df1.iloc[0]['Close'])/df1.iloc[0]['Close']
    print("kosdaq 150 return(",int(year[i+1]),') :',kosdaq_return)
    cumulative_kosdaq_return*=kosdaq_return+1
print('cumulative_kosdaq_return :',cumulative_kosdaq_return-1)  
print('kosdaq_return(CAGR) :',cumulative_kosdaq_return**(1/3)-1  )

kosdaq 150 return( 20 ) : 0.5014854426619133
kosdaq 150 return( 21 ) : -0.01185194994371979
kosdaq 150 return( 22 ) : -0.35513767629281395
cumulative_kosdaq_return : -0.04322427545832053
kosdaq_return(CAGR) : -0.01462081832531037
